In [15]:
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset
from torch.nn import functional as F
import torchvision
from torchvision import datasets,transforms
import torchvision.transforms as transforms

In [2]:
pip install optuna

     |████████████████████████████████| 307kB 7.7MB/s 
     |████████████████████████████████| 81kB 11.5MB/s 
     |████████████████████████████████| 174kB 49.6MB/s 
     |████████████████████████████████| 143kB 47.9MB/s 
     |████████████████████████████████| 112kB 44.8MB/s 
     |████████████████████████████████| 51kB 7.9MB/s 
     |████████████████████████████████| 81kB 11.8MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-cp37-none-any.whl size=11136 sha256=b119531b8d22da3c541b6e0451ddff407ec62c72ea08687becb30757dfe7623f
  Stored in directory: /root/.cache/pip/wheels/25/af/b8/3407109267803f4015e1ee2ff23be0c8c19ce4008665931ee1
Successfully built pyperclip


In [4]:
import optuna
import os

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
CLASSES = 10
DIR = os.getcwd()
EPOCHS = 10
LOG_INTERVAL = 10
#N_TRAIN_EXAMPLES = BATCHSIZE * 30
#N_VALID_EXAMPLES = BATCHSIZE * 10

In [3]:
train_dataset = torchvision.datasets.MNIST('classifier_data', train=True, download=True)
m=len(train_dataset)

transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor()
])

train_dataset.transform=transform


Extracting classifier_data/MNIST/raw/train-images-idx3-ubyte.gz to classifier_data/MNIST/raw




Extracting classifier_data/MNIST/raw/train-labels-idx1-ubyte.gz to classifier_data/MNIST/raw




Extracting classifier_data/MNIST/raw/t10k-images-idx3-ubyte.gz to classifier_data/MNIST/raw




Extracting classifier_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to classifier_data/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


## 1 define cnn model with hyperparameters to tune

In [5]:
class ConvNet(nn.Module):
    def __init__(self, trial):
        # We optimize dropout rate in a convolutional neural network.
        super(ConvNet, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2)

        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        dropout_rate = trial.suggest_float("dropout_rate", 0, 0.5,step=0.1)
        self.drop1=nn.Dropout2d(p=dropout_rate)   
        
        fc2_input_dim = trial.suggest_int("fc2_input_dim", 32, 128,32)
        self.fc1 = nn.Linear(32 * 7 * 7, fc2_input_dim)
        dropout_rate2 = trial.suggest_float("dropout_rate2", 0, 0.3,step=0.1)
        self.drop2=nn.Dropout2d(p=dropout_rate2)

        self.fc2 = nn.Linear(fc2_input_dim, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x),kernel_size = 2))

        x = F.relu(F.max_pool2d(self.conv2(x),kernel_size = 2))
        x = self.drop1(x)

        x = x.view(x.size(0),-1)

        x = F.relu(self.fc1(x))
        x = self.drop2(x)

        x = self.fc2(x)

        return x

## 2 function used to change every time the batch size

In [6]:
def get_mnist(train_dataset,batch_size):

    train_data, val_data = random_split(train_dataset, [int(m-m*0.2), int(m*0.2)])
    # The dataloaders handle shuffling, batching, etc...
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
    valid_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size)
    return train_loader, valid_loader

## 3 train model

In [7]:
def objective(trial):

    # Generate the model.
    model = ConvNet(trial).to(DEVICE)

    # Generate the optimizers.

    # try RMSprop and SGD
    '''
    optimizer_name = trial.suggest_categorical("optimizer", ["RMSprop", "SGD"])
    momentum = trial.suggest_float("momentum", 0.0, 1.0)
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr,momentum=momentum)
    '''
    #try Adam, AdaDelta adn Adagrad
    
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "Adadelta","Adagrad"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1,log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    batch_size=trial.suggest_int("batch_size", 64, 256,step=64)

    criterion=nn.CrossEntropyLoss()

    # Get the MNIST imagesset.
    train_loader, valid_loader = get_mnist(train_dataset,batch_size)
    
    # Training of the model.
    for epoch in range(EPOCHS):
        model.train()
       
        for batch_idx, (images, labels) in enumerate(train_loader):
            # Limiting training images for faster epochs.
            #if batch_idx * BATCHSIZE >= N_TRAIN_EXAMPLES:
            #    break

            images, labels = images.to(DEVICE), labels.to(DEVICE)

            optimizer.zero_grad()
            output = model(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()

        # Validation of the model.
        model.eval()
        correct = 0
        with torch.no_grad():
            for batch_idx, (images, labels) in enumerate(valid_loader):
                # Limiting validation images.
               # if batch_idx * BATCHSIZE >= N_VALID_EXAMPLES:
                #    break
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                output = model(images)
                # Get the index of the max log-probability.
                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(labels.view_as(pred)).sum().item()

        accuracy = correct / len(valid_loader.dataset)

        trial.report(accuracy, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return accuracy

In [16]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

trial = study.best_trial

print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2021-07-11 15:08:16,881] A new study created in memory with name: no-name-7fb2e154-bf24-4f70-98de-fb58bdb05ed3
[I 2021-07-11 15:09:00,402] Trial 0 finished with value: 0.10516666666666667 and parameters: {'dropout_rate': 0.2, 'fc2_input_dim': 96, 'dropout_rate2': 0.1, 'optimizer': 'Adam', 'lr': 0.09894474174180204, 'batch_size': 192}. Best is trial 0 with value: 0.10516666666666667.
[I 2021-07-11 15:09:42,560] Trial 1 finished with value: 0.89025 and parameters: {'dropout_rate': 0.4, 'fc2_input_dim': 32, 'dropout_rate2': 0.1, 'optimizer': 'Adadelta', 'lr': 0.00927013552597451, 'batch_size': 256}. Best is trial 1 with value: 0.89025.
[I 2021-07-11 15:10:28,654] Trial 2 finished with value: 0.10441666666666667 and parameters: {'dropout_rate': 0.30000000000000004, 'fc2_input_dim': 128, 'dropout_rate2': 0.1, 'optimizer': 'Adam', 'lr': 0.042242885494400674, 'batch_size': 128}. Best is trial 1 with value: 0.89025.
[I 2021-07-11 15:11:10,756] Trial 3 finished with value: 0.10325 and parame

Accuracy: 0.9891666666666666
Best hyperparameters: {'dropout_rate': 0.0, 'fc2_input_dim': 96, 'dropout_rate2': 0.3, 'optimizer': 'Adagrad', 'lr': 0.024562626549140668, 'batch_size': 128}


In [25]:
df = study.trials_dataframe().drop(['state','datetime_start','datetime_complete','duration','number'], axis=1)
df.tail(5)

,value,params_batch_size,params_dropout_rate,params_dropout_rate2,params_fc2_input_dim,params_lr,params_optimizer
15,0.115000,128,0.4,0.3,96,0.026889,Adam
16,0.868917,64,0.3,0.2,128,0.000538,Adagrad
17,0.801250,128,0.1,0.0,32,0.000042,Adam
18,0.983083,64,0.5,0.2,64,0.004675,Adagrad
19,0.989167,128,0.0,0.3,96,0.024563,Adagrad


## 4 Plot the study

In [17]:
optuna.visualization.plot_optimization_history(study)

## 5 Plot surface for the hyperparameters involved in the CNN model

In [18]:
optuna.visualization.plot_contour(study, params=['batch_size', 'lr'])

## 6 Parallel coordinate plot

In [19]:
optuna.visualization.plot_parallel_coordinate(study)

## 7 Hyperparameter importances

In [20]:
optuna.visualization.plot_param_importances(study)